In [29]:
import pandas as pd
import datetime as dt
import numpy as np

In [30]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202537.csv")

In [31]:
ema = ema.copy()

# -------------------------------
# 1. Calcular venta_costo_ema
# -------------------------------
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# -------------------------------
# 2. Totales por sucursal
# -------------------------------
ema["venta_costo_tot"] = ema.groupby("store_name")["venta_costo_ema"].transform("sum")
ema["total_ema"] = ema.groupby("store_name")["EMA"].transform("sum")

# -------------------------------
# 3. Proporciones dentro de cada sucursal
# -------------------------------
ema["venta_costo%"] = (ema["venta_costo_ema"] / ema["venta_costo_tot"]).round(4)
ema["cantidad%"] = (ema["EMA"] / ema["total_ema"]).round(4)

# -------------------------------
# 4. Orden y acumulados
# -------------------------------
# Ordenar por costo dentro de cada sucursal
ema = ema.sort_values(["store_name", "venta_costo_ema"], ascending=[True, False])
ema["acumulado_costo"] = ema.groupby("store_name")["venta_costo%"].cumsum().round(4)

# Ordenar por cantidad dentro de cada sucursal
ema = ema.sort_values(["store_name", "EMA"], ascending=[True, False])
ema["acumulado_cantidad"] = ema.groupby("store_name")["cantidad%"].cumsum().round(2)

# Acumulado de desviación por sucursal
if "desviacion_ema%" in ema.columns:
    ema["acumulado_desviacion"] = (
        ema.groupby("store_name")["desviacion_ema%"].cumsum().round(2)
    )

# -------------------------------
# 5. Renombrar columnas (estilo reporte Odoo)
# -------------------------------
ema_def = ema.rename(
    columns={
        "año": "Año",
        "semana_num": "semana",
        "EMA": "Ema",
        "producto_costo_unitario": "Costo unitario",
        "venta_costo_ema": "Venta costo EMA",
        "venta_costo%": "Venta costo %",
        "acumulado_costo": "Acumulado costo",
        "desviacion_ema%": "Desviacion EMA %",
    }
)

# -------------------------------
# 6. Calcular desviación en dinero
# -------------------------------
if "Desviacion EMA %" in ema_def.columns:
    ema_def["Desviacion EMA"] = (
        ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]
    ).round(1)



# Ejemplo de reglas (lo que viene de conf_acumulado_desviacion en Odoo) #PERILLA
reglas = [
    {"rango_ini": 0, "rango_fin": 0.8, "valor": 0},
    {"rango_ini": 0.8, "rango_fin": 1.5, "valor": 0.5},
    {"rango_ini": 1.5, "rango_fin": 3, "valor": 0.8},
    {"rango_ini": 3, "rango_fin": 2000, "valor": 1},
]

# aplicar reglas 0–0.8–1 como antes, pero sobre coef_ventas_norm


# Función que asigna el valor según reglas
def asignar_desviacion(coef, reglas):
    for r in reglas:
        if r["rango_ini"] <= coef <= r["rango_fin"]:
            return r["valor"]
    return np.nan  # si no entra en ningún rango

# Aplicar
ema_def["acumulado_desviacion_p"] = ema_def["coef_ventas"].apply(
    lambda x: asignar_desviacion(x, reglas)
)



# -------------------------------
# 7. Limpiar columnas innecesarias
# -------------------------------
if "Unnamed: 0" in ema_def.columns:
    ema_def = ema_def.drop(columns=["Unnamed: 0"])


# 8. Reglas acumulado costo combinado
# --- Configuración ---
def check_div(a, b):
    try:
        return a / b if b != 0 else 0
    except Exception:
        return 0

## PERILLA

conf_porcentajes_impacto = {
    "cantidad": 0.5,
    "costo": 0.3,
    "desviacion": 0.2,
    "cantidad_mayor": 0.7,
    "costo_mayor": 0.3,
}

# Valor de referencia en porcentaje (ejemplo: 10%) ##PERILLA
conf_acumulado_cantidad = 80
conf_acumulado_cantidad_desviacion = check_div(conf_acumulado_cantidad, 100)


# --- Cálculo en el DataFrame ---
def calcular_combinado(row):
    acumulado_cantidad_final = round(row["acumulado_cantidad"], 2)
    acumulado_costo_final = round(row["Acumulado costo"], 2)
    acumulado_desviacion_p_final = round(row["acumulado_desviacion_p"], 2)

    if acumulado_cantidad_final >= conf_acumulado_cantidad_desviacion:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad_mayor"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo_mayor"]
        )
    else:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo"]
            + acumulado_desviacion_p_final * conf_porcentajes_impacto["desviacion"]
        )

# Nueva columna en tu df
ema_def["acumulado_combinado"] = ema_def.apply(calcular_combinado, axis=1).round(2)


def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.8:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

ema_def["Clasificacion"] = ema_def["acumulado_combinado"].apply(clasificar_pareto)


In [32]:
ema_ord = ema_def[["store_name","product_ref","Año","semana","Ema",
"Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo","Desviacion EMA",
"Desviacion EMA %","acumulado_cantidad","coef_ventas","acumulado_desviacion","acumulado_desviacion_p","acumulado_combinado","Clasificacion"]]

ema_ord.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv")

bq = ema_ord[ema_ord["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False)

In [33]:
bq.head((5))

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion
15169,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,37,38.554979,13507.71,520789.48,0.0084,0.1579,98950.0,0.19,0.02,3.935484,0.19,1.0,0.26,AAA
12686,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,37,33.981446,32797.97,1114522.45,0.0181,0.0954,122597.5,0.11,0.04,1.585421,0.30,0.8,0.21,AAA
17934,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,37,30.869047,9774.98,301744.32,0.0049,0.3099,190098.9,0.63,0.06,5.621262,0.93,1.0,0.32,AAA
15345,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,37,30.471803,10680.97,325468.41,0.0053,0.3050,55329.6,0.17,0.07,3.096774,1.10,1.0,0.32,AAA
15241,SUCURSAL BARRANQUILLA,DAB02666025,2025.0,37,25.581842,11371.65,290907.75,0.0047,0.3337,31999.9,0.11,0.09,2.157303,1.21,0.8,0.30,AAA


In [34]:
df_real = pd.read_csv("/home/donsson/proyectos/API/real_ema.csv")

In [35]:
df_real= df_real[["store_name","almacen_name","product_name","semana","ema","acumulado_combinado","clasificacion"]]
df_mio = ema_ord[["store_name","product_ref","semana","Ema","acumulado_combinado","Clasificacion"]]


In [36]:
conteo = pd.crosstab(df_mio["store_name"], df_mio["Clasificacion"])
conteo

Clasificacion,A,AAA,B,C
store_name,,,,
0,0,0,0,580
PRINCIPAL COTA,141,26,240,3877
SUCURSAL BARRANQUILLA,202,47,361,3674
SUCURSAL BUCARAMANGA,195,44,308,3737
SUCURSAL CALI,161,28,299,3796
SUCURSAL CALLE 6,193,55,352,3684
SUCURSAL MEDELLIN,136,36,312,3800
SUCURSAL NORTE,146,31,307,3800
SUCURSAL VALLADOLID,135,26,278,3845


In [37]:
contar = pd.crosstab(df_real["almacen_name"], df_real["clasificacion"])
contar

clasificacion,A,AAA,B,C
almacen_name,,,,
ANTIOQUEÑA DE LUBRICANTES SGP SAS,24,5,37,4796
BARRANQUILLA,210,41,384,4227
BUCARAMANGA,208,39,319,4296
CALI,153,28,291,4390
CALLE 6,199,55,366,4242
MEDELLIN,143,34,314,4371
MOSTRADOR COTA,119,18,281,4444
NORTE,168,26,298,4370
VALLADOLID,149,29,280,4404
